### Minimal Example for a Hybrid Job using a function decorator ``hybrid_job``

In [1]:
# Import of own modules
from needed_files.quantumenvironment import QuantumEnvironment
from needed_files.q_env_config import q_env_config as gate_q_env_config

### INFO ###
# The following modules would also be used in the real implementation but not relevant for this mininal example
# Both load_agent_from_yaml_file and make_train_ppo are functions
# from needed_files.helper_functions import load_agent_from_yaml_file
# from needed_files.ppo import make_train_ppo


# Braket imports
from braket.tracking import Tracker
from braket.jobs import hybrid_job
from braket.aws import AwsSession
from qiskit_braket_provider import AWSBraketProvider

# Specify the S3 bucket to store circuit and result files
aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

# Other imports
import time

/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(
I0000 00:00:1705032647.467365       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


Find below the function that shall later be converted into a hybrid job with a decorator. We tested the full code (below only a scheme to understand the workflow) **without** a decorator and it works well. 
Once, the decorator is applied, we face issues with the serialization of one of the objects we need to use for the simulation. This object is ``gate_q_env_config`` that gets imported from a module in the directory "needed_files".

Said object contains all necessary information about the simulation and is therefore crucial element since it will be used as input by the ``QuantumEnvironment`` class as the first step of our workflow.

In [2]:
def calibrate_gate():
    print("Job started!!!!!")
    braket_task_costs = Tracker().start()

    # XXX: The following step, so using the object gate_q_env_config fails because it is non-serializable.
    # q_env = QuantumEnvironment(gate_q_env_config)


    ### INFO ###

    # The code below is a dummy/schematic implementation of what happens later in our workflow.
    # The error-causing line is the one with XXX above. Since this error-causing line is the first step in our workflow,
    # we cannot/could not proceed with the rest of the workflow.

    # Since our codebase is based on Qiskit, we want to use Braket's Qiskit Provider (here start with SV1 for testing)
    provider = AWSBraketProvider()
    backend = provider.get_backend('SV1')
    # q_env.backend = backend
    # Load the RL agent based on the QuantumEnvironment object
    # Run the parametrized quantum circuit that is part of the ``gate_q_env_config`` object
    # and train the RL agent based on the reward
    
    # Return training results (here dummy results)
    dummy_training_results = {
        'avg_return': [0.34, 0.45, 0.56, 0.67, 0.78, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95],
        'action_vector': [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09],
        'task_summary': braket_task_costs.quantum_tasks_statistics(),
        'estimated cost': float(
            braket_task_costs.qpu_tasks_cost() + braket_task_costs.simulator_tasks_cost()
        )
    }
    return dummy_training_results

In [3]:
job_name = f'cx-gate-calibration-decorator-{str(int(time.time()))}'

needed_modules = [
    # This module contains the QuantumEnvironment class which takes the gate_q_env_config object as input
    'needed_files.quantumenvironment',
    # XXX: This module contains the gate_q_env_config object that we want to use and for which the serialization fails
    'needed_files.q_env_config',

    # INFO: The following modules are needed for the real implementation of the calibration but not relevant for the dummy implementation
    # 'needed_files.helper_functions', 
    # 'needed_files.ppo', 
]

Make the function ``calibrate_gate`` a hybrid job with the decorator specifying required additonal information like input data, dependencies and - most importantly - modules

In [4]:
@hybrid_job(device=None, 
            local=False, 
            include_modules=needed_modules, # XXX: This is the list of modules that are needed for the job
            # input_data='config_yamls/agent_config.yaml', Passing the yaml works well, but is not needed for the minimal example
            job_name=job_name, 
            dependencies='requirements.txt')
def calibrate_gate_hybrid_job():
    return calibrate_gate()

In [5]:
job = calibrate_gate_hybrid_job()

Progress Tracker

In [6]:
while job.state() not in job.TERMINAL_STATES:
    print(job.state())
    time.sleep(30)

print(job.state())
print(job.result())

QUEUED
RUNNING
RUNNING
RUNNING
RUNNING
FAILED
FAILED
{}
